<a href="https://colab.research.google.com/github/BrunoPLemos/Assistente_de_Voz_Inteligente_com_Gemini_-_Whisper/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# --- 1. INSTALAÇÃO DAS DEPENDÊNCIAS (Incluindo pydub e ffmpeg para velocidade) ---
!pip install -q -U google-generativeai git+https://github.com/openai/whisper.git gTTS pydub
!apt-get install -y ffmpeg -q

import os
import re
import whisper
import google.generativeai as genai
from base64 import b64decode
from gtts import gTTS
from pydub import AudioSegment
from google.colab import userdata, output
from IPython.display import Javascript, display, HTML, Audio, clear_output

# --- 2. CONFIGURAÇÃO DA IA ---
genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
model_gemini = genai.GenerativeModel('gemini-2.5-flash')
model_whisper = whisper.load_model("base")

# --- 3. INTERFACE E LÓGICA DE GRAVAÇÃO (JavaScript) ---
JS_CODE = """
window.recordAudio = async (time) => {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);
  const chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
  await new Promise(r => setTimeout(r, time));
  return new Promise(resolve => {
    recorder.onstop = async () => {
      const blob = new Blob(chunks);
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      reader.onloadend = () => resolve(reader.result);
    };
    recorder.stop();
  });
};
"""

def exibir_interface():
    html_ui = """
    <div style="font-family: sans-serif; text-align: center; padding: 20px; border: 2px solid #1a73e8; border-radius: 20px; background: #fff; max-width: 320px; margin: auto; box-shadow: 0 10px 20px rgba(0,0,0,0.1);">
        <h2 style="color: #1a73e8; margin-bottom: 10px;">Assistente Rápido</h2>
        <p id="status-text" style="color: #5f6368; margin-bottom: 20px;">Pronto para ouvir</p>
        <button id="record-btn" style="background-color: #1a73e8; color: white; border: none; width: 90px; height: 90px; border-radius: 50%; font-size: 35px; cursor: pointer; transition: 0.3s; outline: none;">
            🎤
        </button>
    </div>

    <script>
    const btn = document.getElementById('record-btn');
    const status = document.getElementById('status-text');

    btn.onclick = async () => {
        btn.disabled = true;
        btn.style.backgroundColor = "#ea4335";
        status.innerText = "🔴 Ouvindo...";

        try {
            const audioBase64 = await window.recordAudio(5000); // 5 segundos
            status.innerText = "⌛ Processando...";
            btn.style.backgroundColor = "#fbbc05";
            google.colab.kernel.invokeFunction('notebook.ProcessarAudio', [audioBase64], {});
        } catch (e) {
            status.innerText = "❌ Erro no microfone";
        } finally {
            setTimeout(() => {
                btn.disabled = false;
                btn.style.backgroundColor = "#1a73e8";
                status.innerText = "Clique para falar";
            }, 4000);
        }
    };
    </script>
    """
    display(Javascript(JS_CODE))
    display(HTML(html_ui))

# --- 4. FUNÇÃO DE PROCESSAMENTO (Com Limpeza e Velocidade) ---
def processar_audio(audio_base64):
    try:
        # 1. Salva áudio original
        audio_bytes = b64decode(audio_base64.split(',')[1])
        with open('pergunta.wav', 'wb') as f: f.write(audio_bytes)

        # 2. Transcrição
        res = model_whisper.transcribe('pergunta.wav', fp16=False, language='pt')
        prompt = res["text"]
        print(f"\n👤 Você: {prompt}")

        # 3. Resposta Gemini
        response = model_gemini.generate_content(prompt)
        texto_raw = response.text

        # --- AJUSTE 1: Limpeza de asteriscos e markdown ---
        texto_limpo = texto_raw.replace('*', '')
        texto_limpo = re.sub(r'#+', '', texto_limpo) # Remove títulos (###)

        print(f"🤖 IA: {texto_raw}")

        # 4. Voz (gTTS)
        tts = gTTS(text=texto_limpo, lang='pt', slow=False)
        tts.save("lento.mp3")

        # --- AJUSTE 2: Aumentar velocidade com Pydub ---
        audio_segment = AudioSegment.from_file("lento.mp3")
        # 1.2 é 20% mais rápido que o normal. Pode ajustar para 1.4 se quiser mais rápido.
        audio_fast = audio_segment.speedup(playback_speed=1.2)
        audio_fast.export("final.mp3", format="mp3")

        display(Audio("final.mp3", autoplay=True))

    except Exception as e:
        print(f"❌ Erro: {e}")

# Registro e Início
output.register_callback('notebook.ProcessarAudio', processar_audio)
clear_output()
exibir_interface()

<IPython.core.display.Javascript object>


👤 Você:  Explique em poucas palavras o que é a inteligência artificial.
🤖 IA: É a capacidade de **máquinas** de **simular a inteligência humana**, ou seja, **pensar, aprender e raciocinar** para **resolver problemas e tomar decisões**.



👤 Você:  explique em poucas palavras o que é ciências de dados.
🤖 IA: É a área que usa dados para **entender o passado, prever o futuro e tomar decisões mais inteligentes.**
